# Step1: create dataset

In [1]:
from pyhealth.datasets import MIMIC3BaseDataset, MIMIC4BaseDataset, eICUBaseDataset, OMOPBaseDataset
base_dataset = MIMIC3BaseDataset(root="/srv/local/data/physionet.org/files/mimiciii/1.4")
# base_dataset = eICUBaseDataset(root="/srv/local/data/physionet.org/files/eicu-crd/2.0")
# base_dataset = MIMIC4BaseDataset(root="/srv/local/data/physionet.org/files/mimiciv/2.0/hosp")
# base_dataset = OMOPBaseDataset(root="/srv/local/data/zw12/pyhealth/raw_data/synpuf1k_omop_cdm_5.2.2")
base_dataset.info()

/home/pj20/miniconda3/envs/pyhealth/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



        ----- Output Data Structure -----
        Dataset.patients: [
            {
                patient_id: patient_id, 
                visits: [
                    {
                        visit_id: visit_id, 
                        patient_id: patient_id, 
                        conditions: [List], 
                        procedures: [List],
                        drugs: [List],
                        visit_info: <dict>
                    }
                    ...
                ]                    
            } 
            ...
        ]
        


user can use this module for data processing
- **[researchers from CS]** build their own model on top of it
- **[reserachers from medical area]** use the models in the package

In [2]:
# create task-specific dataset
from pyhealth.tasks import DrugRecDataset
drug_rec_dataset = DrugRecDataset(base_dataset)
drug_rec_dataset.info()


        ----- Output Data Structure -----
        >> drug_rec_dataloader[0]
        >> {
            "conditions": List[tensor],
            "procedures": List[tensor],
            "drugs": List[tensor]
        }
        


# Step2: healthcare predictive model

In [4]:
voc_size = drug_rec_dataset.voc_size
params = drug_rec_dataset.params

from pyhealth.models import RETAIN
model = RETAIN(voc_size, params)

# Step 3: create train / val / test dataloader

In [5]:
from pyhealth.data import split
from torch.utils.data import DataLoader

drug_rec_trainset, drug_rec_valset, drug_rec_testset = split.random_split(drug_rec_dataset, [0.8, 0.1, 0.1])
drug_rec_train_loader = DataLoader(drug_rec_trainset, batch_size=1, collate_fn=lambda x: x[0])
drug_rec_val_loader = DataLoader(drug_rec_valset, batch_size=1, collate_fn=lambda x: x[0])
drug_rec_test_loader = DataLoader(drug_rec_testset, batch_size=1, collate_fn=lambda x: x[0])

# Step 4: model training and evaluation

In [6]:
# train
from pytorch_lightning import Trainer

trainer = Trainer(
    gpus=1,
    max_epochs=3,
    progress_bar_refresh_rate=5,
)

trainer.fit(
    model=model,
    train_dataloaders=drug_rec_train_loader,
    val_dataloaders=drug_rec_val_loader,
)

/home/pj20/miniconda3/envs/pyhealth/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:96: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=5)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                | Type    

Sanity Checking DataLoader 0:   0%|                                                    | 0/2 [00:00<?, ?it/s]

/home/pj20/miniconda3/envs/pyhealth/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/home/pj20/miniconda3/envs/pyhealth/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:  89%|████████████████████████████████▉    | 4359/4903 [00:40<00:05, 107.39it/s, loss=0.114, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 1:  89%|█████████████████████████████████▊    | 4359/4903 [01:19<00:09, 54.72it/s, loss=0.103, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 2:  89%|█████████████████████████████████▊    | 4359/4903 [01:57<00:14, 37.03it/s, loss=0.107, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 2: 100%|██████████████████████████████████████| 4903/4903 [01:59<00:00, 41.04it/s, loss=0.107, v_num=0]


In [7]:
# evaluation
from pyhealth.evaluator import DrugRecEvaluator
evaluator = DrugRecEvaluator(model)
evaluator.evaluate(drug_rec_test_loader)


Jaccard: 0.132,  PRAUC: 0.3148, AVG_PRC: 0.5275, AVG_RECALL: 0.1575, AVG_F1: 0.2291, AVG_MED: 14.08

